In [1]:
import json
import base64
import io
from PIL import Image
import matplotlib.pyplot as plt

# Path to your log file
LOG_FILE = "../output/logs_kimi.jsonl" # or logs_qwen.jsonl

def decode_image_from_log(base64_string):
    if base64_string.startswith("data:image"):
        base64_string = base64_string.split(",")[1]
    image_data = base64.b64decode(base64_string)
    return Image.open(io.BytesIO(image_data))

# 1. Read the last line of the log
last_line = None
with open(LOG_FILE, 'r') as f:
    for line in f:
        last_line = line

if last_line:
    entry = json.loads(last_line)
    print(f"Token: {entry['token']}")
    print(f"Usage: {entry.get('usage')}")
    
    # 2. Extract Images from the saved Prompt Messages
    # Note: If you used _sanitize_for_logging, the base64 is removed.
    # IF YOU DID NOT sanitize (or if you want to test live), we need to check.
    
    # If your log file has <BASE64_IMAGE_DATA_REMOVED>, we cannot recover it from disk.
    # We must rely on the print statements added in Step 1.
    
    messages = entry.get('prompt_messages', [])
    found_images = False
    
    for msg in messages:
        if isinstance(msg.get('content'), list):
            for content_part in msg['content']:
                if content_part.get('type') == 'image_url':
                    url = content_part['image_url']['url']
                    if "BASE64" in url:
                        print("⚠️ Images were sanitized in logs (Good for disk space, bad for debug).")
                        print("👉 Use the print debug method in Step 1 instead.")
                        found_images = True
                        break
                    else:
                        # Recover Image
                        img = decode_image_from_log(url)
                        print(f"Found Image! Size: {img.size}")
                        plt.imshow(img)
                        plt.show()
                        found_images = True
    
    if not found_images:
        print("No image data found in logs.")

Token: abf3d91d3c28407e80e3334fe89c03cb
Usage: {'input_tokens': 1010, 'output_tokens': 1089, 'total_tokens': 2099}
⚠️ Images were sanitized in logs (Good for disk space, bad for debug).
👉 Use the print debug method in Step 1 instead.
